In [1]:
!pip install praw

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 kB 3.7 MB/s eta 0:00:00


# more data

In [ ]:
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=UserWarning, module="praw")

import logging

# Suppress PRAW warnings
logging.getLogger("praw").setLevel(logging.CRITICAL)


import praw
import pandas as pd
import time

# Reddit API credentials
reddit = praw.Reddit(client_id='fgsA4XN83I9FEZdiZvRDtw',
                     client_secret='WLahZdE7ybEoPcIgN-MuMSqWurCDHQ',
                     user_agent='Mental Health Scraper')

# Search for subreddits containing 'ptsd'
# query = 'bipolar'  # Substring to search for
# matching_subreddits = []
# for subreddit in reddit.subreddits.search(query, limit=None):
#     if 'bipolar' in subreddit.display_name.lower():  # Case-insensitive check
#         matching_subreddits.append(subreddit.display_name)

# List of keywords to exclude from subreddit names
exclude_keywords = ['anxiety', 'bipolar', 'ptsd', 'depression','normal', 'news', 'technology', 'gaming', 'movies', 'books', 'AskReddit', 'worldnews']
# Initialize a list to store the matching subreddits
matching_subreddits = []
# Set a reasonable limit for the number of subreddits to search
limit = 100  # Adjust this limit as needed
# Perform the search for subreddits
for submission in reddit.subreddits.popular(limit=limit):  # Use `popular` to get a list of popular subreddits
    if all(keyword not in submission.display_name.lower() for keyword in exclude_keywords):
        matching_subreddits.append(submission.display_name)
# Limit to the top 15 subreddits / change the indices as per need
print(len(matching_subreddits))
# matching_subreddits = matching_subreddits[50:]
print(f"Found subreddits: {matching_subreddits}")

# Number of posts to scrape per subreddit and type
total_posts_per_subreddit = 20000
posts_per_type = total_posts_per_subreddit  # Divide among 'hot', 'new', 'top'

# List to store all posts
all_data = []

# Scrape posts from each matching subreddit
post_types = ['hot', 'new', 'top']

num=1
count=0

try:
    for subreddit_name in matching_subreddits:
        print(f"Scraping subreddit: {subreddit_name}")
        subreddit = reddit.subreddit(subreddit_name)
        count=0
        for post_type in post_types:
            if post_type == 'hot':
                subreddit_posts = subreddit.hot(limit=posts_per_type)
            elif post_type == 'new':
                subreddit_posts = subreddit.new(limit=posts_per_type)
            elif post_type == 'top':
                subreddit_posts = subreddit.top(limit=posts_per_type)

            for post in subreddit_posts:
                post_content = post.title + " " + post.selftext  # Combine title and body
                all_data.append([post_content, 'normal'])  # Add text, category, subreddit name
                count+=1

            # Avoid hitting Reddit's rate limit
            time.sleep(2)

        print(f"{num}. Total Scraped {len(all_data)}. From {subreddit_name} is {count}")
        num+=1

except Exception as e:
    pass
finally:
    # Convert collected data to a DataFrame
    df = pd.DataFrame(all_data, columns=['text', 'mental_health_issue'])
    # Save to CSV
    output_file = 'all_normal2.csv'
    df.to_csv(output_file, index=False)

    # Display the first 10 rows
    print(f"Data saved to {output_file}")
    print(df.head(10))


# Data Cleaning

In [74]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk

# Download stopwords (if you haven't already)
nltk.download('stopwords')
nltk.download('punkt')
# Download the 'punkt_tab' resource
nltk.download('punkt_tab')  # This line is added to download the necessary data

# Load the dataset
df = pd.read_csv('final_normal.csv')

# 1. Handling Missing Values
# Remove rows with missing text
df.dropna(subset=['text'], inplace=True)

# 2. Removing duplicates (if any)
df.drop_duplicates(subset=['text'], inplace=True)

# 3. Text Preprocessing

# Define a list of negative words to retain
negative_words = {"not", "no", "nor", "never", "nothing", "nowhere", "neither", "cannot", "n't", "without", "barely", "hardly", "scarcely"}

# Define a function to clean the text
def clean_text(text):
    # Remove URLs
    text = re.sub(r'http\S+', '', text)

    # Remove mentions (@username)
    text = re.sub(r'@\w+', '', text)

    # Remove special characters, numbers, and punctuations
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Convert text to lowercase
    text = text.lower()

    # Tokenize the text
    tokens = word_tokenize(text)

    # Remove stopwords, but keep negative words
    tokens = [word for word in tokens if word not in stopwords.words('english') or word in negative_words]

    # Join the tokens back into a single string
    clean_text = ' '.join(tokens)

    return clean_text

# Apply the cleaning function to the 'text' column
df['cleaned_text'] = df['text'].apply(clean_text)

# Save the preprocessed dataset (optional)
df.to_csv('final_preprocessed_normal.csv', index=False)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


# Combine all datasets

In [73]:
import pandas as pd

# File paths for the two datasets
file1 = 'all_normal.csv'
file2 = 'all_normal2.csv'

# Read the datasets
df1 = pd.read_csv(file1)
df2 = pd.read_csv(file2)

# Combine the datasets
combined_df = pd.concat([df1, df2], ignore_index=True)

# Save the combined dataset to a new CSV file
output_file = 'final_normal.csv'
combined_df.to_csv(output_file, index=False)

# Display the number of records and the first few rows
print(f"Combined dataset saved as {output_file} with {len(combined_df)} records.")
print(combined_df.head())


Combined dataset saved as final_normal.csv with 131725 records.
                                                text mental_health_issue
0  Rules **UPDATED** 1. If you're here you must p...              normal
1  Happy Cakeday, r/Normal! Today you're 11 Let's...              normal
2  Happy Cakeday, r/Normal! Today you're 10 Let's...              normal
3  I am also a person I am very normal. Today I t...              normal
4  Happy Cakeday, r/Normal! Today you're 9 Let's ...              normal


In [70]:
import pandas as pd
import os

# Get all CSV files in the current directory
csv_files = [f for f in os.listdir() if f.endswith('.csv')]

# Function to combine datasets and save to a new CSV
def combine_and_save(file_list, output_filename):
    # Read and combine all CSV files
    df_list = [pd.read_csv(file) for file in file_list]
    combined_df = pd.concat(df_list, ignore_index=True)

    # Save to a new CSV file
    combined_df.to_csv(output_filename, index=False)

    # Return the count of records
    return len(combined_df)

# Combine files related to anxiety
anxiety_files = [f for f in csv_files if 'all_anxiety' in f]
anxiety_count = combine_and_save(anxiety_files, 'final_anxiety.csv')
print(f"Combined anxiety files: {anxiety_count} records")

# Combine files related to PTSD
ptsd_files = [f for f in csv_files if 'all_ptsd' in f]
ptsd_count = combine_and_save(ptsd_files, 'final_ptsd.csv')
print(f"Combined PTSD files: {ptsd_count} records")

# Combine files related to depression
depression_files = [f for f in csv_files if 'all_depression' in f]
depression_count = combine_and_save(depression_files, 'final_depression.csv')
print(f"Combined depression files: {depression_count} records")

# Combine files related to bipolar
bipolar_files = [f for f in csv_files if 'all_bipolar' in f]
bipolar_count = combine_and_save(bipolar_files, 'final_bipolar.csv')
print(f"Combined bipolar files: {bipolar_count} records")

# Combine files related to preprocessed anxiety
preprocessed_anxiety_files = [f for f in csv_files if 'preprocessed_anxiety' in f]
preprocessed_anxiety_count = combine_and_save(preprocessed_anxiety_files, 'final_preprocessed_anxiety.csv')
print(f"Combined preprocessed anxiety files: {preprocessed_anxiety_count} records")

# Combine files related to preprocessed depression
preprocessed_depression_files = [f for f in csv_files if 'preprocessed_depression' in f]
preprocessed_depression_count = combine_and_save(preprocessed_depression_files, 'final_preprocessed_depression.csv')
print(f"Combined preprocessed depression files: {preprocessed_depression_count} records")

# Combine files related to preprocessed bipolar
preprocessed_bipolar_files = [f for f in csv_files if 'preprocessed_bipolar' in f]
preprocessed_bipolar_count = combine_and_save(preprocessed_bipolar_files, 'final_preprocessed_bipolar.csv')
print(f"Combined preprocessed bipolar files: {preprocessed_bipolar_count} records")

# Combine files related to preprocessed PTSD
preprocessed_ptsd_files = [f for f in csv_files if 'preprocessed_ptsd' in f]
preprocessed_ptsd_count = combine_and_save(preprocessed_ptsd_files, 'final_preprocessed_ptsd.csv')
print(f"Combined preprocessed PTSD files: {preprocessed_ptsd_count} records")


Combined anxiety files: 38580 records
Combined PTSD files: 34019 records
Combined depression files: 36908 records
Combined bipolar files: 30420 records
Combined preprocessed anxiety files: 20456 records
Combined preprocessed depression files: 19446 records
Combined preprocessed bipolar files: 15746 records
Combined preprocessed PTSD files: 16386 records


In [1]:
import pandas as pd

# File path for the dataset
file = 'final_preprocessed_normal.csv'

# Read the dataset
df = pd.read_csv(file)

# Get the first 40,000 records
first_40k_df = df.head(40000)

# Save the new dataset to a CSV file
output_file = 'final_preprocessed_normal1.csv'
first_40k_df.to_csv(output_file, index=False)

# Display the number of records and the first few rows
print(f"New dataset with first 40,000 records saved as {output_file}.")
print(first_40k_df.head())


New dataset with first 40,000 records saved as final_preprocessed_normal1.csv.
                                                text mental_health_issue  \
0  Rules **UPDATED** 1. If you're here you must p...              normal   
1  Happy Cakeday, r/Normal! Today you're 11 Let's...              normal   
2  Happy Cakeday, r/Normal! Today you're 10 Let's...              normal   
3  I am also a person I am very normal. Today I t...              normal   
4  Happy Cakeday, r/Normal! Today you're 9 Let's ...              normal   

                                        cleaned_text  
0  rules updated youre must post something normal...  
1  happy cakeday rnormal today youre lets look ba...  
2  happy cakeday rnormal today youre lets look ba...  
3  also person normal today took dogs walk park a...  
4  happy cakeday rnormal today youre lets look ba...  


In [2]:
import pandas as pd

# List of all the CSV file names
csv_files = ['final_preprocessed_normal1.csv', 'final_preprocessed_ptsd.csv', 'final_preprocessed_anxiety.csv',
             'final_preprocessed_depression.csv', 'final_preprocessed_bipolar.csv']

# Display the number of records in each CSV file
for file in csv_files:
    df = pd.read_csv(file)
    print(f"Number of records in {file}: {len(df)}")

# Read and combine all CSV files
df_list = [pd.read_csv(file) for file in csv_files]
combined_df = pd.concat(df_list, ignore_index=True)

# Display the number of records in the final combined CSV
print(f"Number of records in the combined CSV before shuffling: {len(combined_df)}")

# Shuffle the rows to jumble them up
shuffled_df = combined_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Save the combined and shuffled dataset to a new CSV
shuffled_df.to_csv('preprocessed_mental_health.csv', index=False)

# Display the number of records in the final shuffled CSV
print(f"Number of records in the shuffled CSV: {len(shuffled_df)}")

# Display the first few rows
print(shuffled_df.head(10))


Number of records in final_preprocessed_normal1.csv: 40000
Number of records in final_preprocessed_ptsd.csv: 16386
Number of records in final_preprocessed_anxiety.csv: 20456
Number of records in final_preprocessed_depression.csv: 19446
Number of records in final_preprocessed_bipolar.csv: 15746
Number of records in the combined CSV before shuffling: 112034
Number of records in the shuffled CSV: 112034
                                                text mental_health_issue  \
0  Do you ever just wait for someone to give up o...          depression   
1  Interesting tips here on effectively observing...                ptsd   
2  Question for a choice to make on a video I've ...              normal   
3           What motivates you to better yourself?                normal   
4  So almost a week has gone since the allegation...              normal   
5  Battle perfectionism with these six weird phra...             anxiety   
6  Scared for freeway drive Hey, I (16f) posted h...            